<a href="https://colab.research.google.com/github/jcmachicao/deep_learning_2025_curso/blob/main/S05__rnn_lluvias_a%C3%B1o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN Seq2Seq: Predecir 30 días de precipitación (PyTorch)
### Notebook preparado para Google Colab / ejecución local.

In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
# ------------------------------
# 0. Config
# ------------------------------
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ------------------------------
# 1. Dataset sintético (estacional + ruido)
# ------------------------------
# Queremos generar muchas series anuales (365 días) y predecir los próximos 30 días.
INPUT_LEN = 365
TARGET_LEN = 30
SAMPLES = 1200  # cantidad de ejemplos sintéticos

# Generador: temporada anual + pequeñas variaciones por "año"
def generate_yearly_precipitation(seed_offset=0):
    # Base estacional: mezcla de senos para simular estaciones y eventos
    days = np.arange(INPUT_LEN + TARGET_LEN)
    # componente anual (estacionalidad fuerte)
    annual = 10.0 * (np.sin(2 * np.pi * days / 365 - 0.2) + 1.0)
    # componente subanual (meses húmedos intermitentes)
    monthly = 3.0 * np.sin(2 * np.pi * days / 30 + 0.5)
    # tendencia aleatoria por "año"
    rnd = 2.0 * np.random.randn(INPUT_LEN + TARGET_LEN) * 0.5
    # pequeños picos de lluvia aleatorios
    spikes = np.where(np.random.rand(INPUT_LEN + TARGET_LEN) > 0.98, np.random.rand(INPUT_LEN + TARGET_LEN) * 20, 0.0)

    series = np.maximum(0.0, annual + monthly + rnd + spikes)
    # añadir pequeña variación dependiente del offset para crear diversidad entre series
    series *= (1.0 + 0.05 * np.sin(seed_offset * 0.37))
    return series.astype(np.float32)

# Construir dataset con ventana (cada ejemplo: 365 in -> 30 out)
X = []
Y = []
for i in range(SAMPLES):
    series = generate_yearly_precipitation(i)
    x = series[:INPUT_LEN]
    y = series[INPUT_LEN:INPUT_LEN+TARGET_LEN]
    # normalizar por serie (opcional): media/std
    mu = x.mean()
    sigma = x.std() + 1e-6
    x_norm = (x - mu) / sigma
    y_norm = (y - mu) / sigma
    X.append(x_norm[:, None])  # shape (365, 1)
    Y.append(y_norm[:, None])  # shape (30, 1)

X = np.stack(X)  # (SAMPLES, 365, 1)
Y = np.stack(Y)  # (SAMPLES, 30, 1)

# dividir train/val/test
n_train = int(0.8 * SAMPLES)
X_train, Y_train = X[:n_train], Y[:n_train]
X_val, Y_val = X[n_train:int(0.9*SAMPLES)], Y[n_train:int(0.9*SAMPLES)]
X_test, Y_test = X[int(0.9*SAMPLES):], Y[int(0.9*SAMPLES):]

print('Shapes ->', X_train.shape, Y_train.shape, X_val.shape, X_test.shape)

# ------------------------------
# 2. Dataset y DataLoader
# ------------------------------
class TimeSeriesDataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.from_numpy(X).float()
        self.Y = torch.from_numpy(Y).float()
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

BATCH = 32
train_loader = DataLoader(TimeSeriesDataset(X_train, Y_train), batch_size=BATCH, shuffle=True)
val_loader = DataLoader(TimeSeriesDataset(X_val, Y_val), batch_size=BATCH)

# ------------------------------
# 3. Model: Encoder-Decoder RNN simple
# ------------------------------
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super().__init__()
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)
    def forward(self, x):
        # x: (batch, seq_len, input_size)
        out, h_n = self.rnn(x)
        return h_n  # (num_layers, batch, hidden_size)

class DecoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size=1, num_layers=1):
        super().__init__()
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, decoder_inputs, hidden):
        # decoder_inputs: (batch, tgt_len, input_size)
        out, h_n = self.rnn(decoder_inputs, hidden)
        # proyectar cada paso
        out2 = self.fc(out)  # (batch, tgt_len, output_size)
        return out2, h_n

# Modelo completo wrapper
class Seq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, target_len, device):
        super().__init__()
        self.encoder = EncoderRNN(input_size, hidden_size)
        # para decoder usaremos el mismo input_size (1) ya que alimentamos el valor anterior
        self.decoder = DecoderRNN(input_size=1, hidden_size=hidden_size, output_size=1)
        self.target_len = target_len
        self.device = device

    def forward(self, src, tgt=None, teacher_forcing_ratio=0.5):
        # src: (batch, INPUT_LEN, 1)
        batch_size = src.size(0)
        hidden = self.encoder(src)  # (num_layers, batch, hidden)

        # Inicializamos el input del decoder con el último valor del src
        # alternativa: usar un token <sos>. Aquí usamos el último día observado.
        last_day = src[:, -1:, :]  # (batch, 1, 1)
        decoder_input = last_day

        outputs = torch.zeros(batch_size, self.target_len, 1, device=self.device)

        for t in range(self.target_len):
            out, hidden = self.decoder(decoder_input, hidden)
            # out: (batch, 1, 1)
            outputs[:, t:t+1, :] = out

            # determinar el siguiente input: teacher forcing o predicción
            if tgt is not None and random.random() < teacher_forcing_ratio:
                # usar el valor verdadero (tgt) durante entrenamiento
                decoder_input = tgt[:, t:t+1, :]
            else:
                # usar la propia predicción
                decoder_input = out.detach()

        return outputs

# Instanciar
INPUT_SIZE = 1
HIDDEN_SIZE = 64
model = Seq2Seq(INPUT_SIZE, HIDDEN_SIZE, TARGET_LEN, DEVICE).to(DEVICE)

# ------------------------------
# 4. Entrenamiento
# ------------------------------
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
EPOCHS = 20

for epoch in range(1, EPOCHS+1):
    model.train()
    epoch_loss = 0.0
    for xb, yb in train_loader:
        xb = xb.to(DEVICE)
        yb = yb.to(DEVICE)
        optimizer.zero_grad()
        out = model(xb, tgt=yb, teacher_forcing_ratio=0.5)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * xb.size(0)
    epoch_loss /= len(train_loader.dataset)

    # val
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(DEVICE)
            yb = yb.to(DEVICE)
            out = model(xb, tgt=None, teacher_forcing_ratio=0.0)
            val_loss += criterion(out, yb).item() * xb.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch:02d}  Train Loss: {epoch_loss:.6f}  Val Loss: {val_loss:.6f}")

# ------------------------------
# 5. Guardar y cargar pesos
# ------------------------------
torch.save(model.state_dict(), 'seq2seq_rnn.pth')

model2 = Seq2Seq(INPUT_SIZE, HIDDEN_SIZE, TARGET_LEN, DEVICE).to(DEVICE)
model2.load_state_dict(torch.load('seq2seq_rnn.pth', map_location=DEVICE))
model2.eval()

# ------------------------------
# 6. Inferencia y ejemplo gráfico
# ------------------------------
# tomar un ejemplo de test
x_test = torch.from_numpy(X_test[0:1]).float().to(DEVICE)  # (1, 365, 1)
y_true = Y_test[0:1]
with torch.no_grad():
    y_pred = model2(x_test, tgt=None, teacher_forcing_ratio=0.0)

y_pred = y_pred.cpu().numpy().squeeze()
y_true = y_true.squeeze()

# des-normalizar: recordamos que normalizamos por la serie de entrenamiento (en el generator usamos su propia mu/sigma)
# en este notebook normalizamos por serie individual, por lo que la comparativa se mantiene sin revertir.

plt.figure(figsize=(10,4))
plt.plot(np.arange(TARGET_LEN), y_true, label='True (30 días)')
plt.plot(np.arange(TARGET_LEN), y_pred, label='Predicción (30 días)')
plt.legend()
plt.title('Predicción de precipitación - 30 días')
plt.xlabel('Día (horizonte)')
plt.ylabel('Precipitación (norm.)')
plt.grid(True)
plt.show()

print('Predicciones (primeros 10):', y_pred[:10])

# 7. Notas pedagógicas
------------------------------
- Aquí usamos un encoder que resume la serie completa en el estado oculto. Para horizontes largos
  (365->365) esa aproximación puede fallar: conviene usar attention o transformers.
- Utilizamos normalización por serie. En práctica real se normaliza con parámetros del conjunto de entrenamiento.
- Teacher forcing acelera el aprendizaje en decoder, pero produce discrepancias en inferencia (exposure bias).
- Para datasets reales: limpiar, imputar ceros, transformar (log1p) y evaluar métricas de error (RMSE, MAE).